In [24]:
import pandas as pd
from scipy import stats
from scipy.stats import chi2_contingency
import numpy as np

In [2]:
data = pd.read_csv('water.txt', sep='\t')
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [4]:

round(stats.pearsonr(hardness, mortality)[0], 4)

-0.6548

In [5]:
round(stats.spearmanr(hardness, mortality)[0], 4)

-0.6317

In [11]:
mortality_s = data[data['location'] == 'South']['mortality']
hardness_s = data[data['location'] == 'South']['hardness']
p_s = round(stats.pearsonr(hardness_s, mortality_s)[0], 4)

mortality_n = data[data['location'] == 'North']['mortality']
hardness_n = data[data['location'] == 'North']['hardness']
p_n = round(stats.pearsonr(hardness_n, mortality_n)[0], 4)

print(p_s if abs(p_s) < abs(p_n) else p_n )

-0.3686


In [17]:
N = 203 + 239 + 718 + 515
S = (203 + 718) / N
P = (203 + 239) / N
corr_m = (203/N - S*P)/(P*S*(1 - S)*(1 - P))**0.5
print(round(corr_m, 4))

-0.109


In [20]:
chi2_contingency([[203, 718], [239, 515]])

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[243.03402985, 677.96597015],
        [198.96597015, 555.03402985]]))

In [21]:
men = [1] * 239 + [0] * 515
women = [1] * 203 + [0] * 718
len(men), len(women)

(754, 921)

In [25]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [27]:
round(proportions_diff_confint_ind(men, women)[0], 4)

0.0539

In [28]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [30]:
proportions_diff_z_test(proportions_diff_z_stat_ind(men, women))

8.153453089576601e-06

In [34]:
arr = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

chi2, p_val, _, _ = chi2_contingency(arr)
round(chi2, 4)

293.6831

In [33]:
chi2_contingency(arr)[1]

2.4964299580093467e-62